In [449]:
import pymysql
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from glob import glob

## Q at V

In [447]:
datafilespath = glob('EIS/raw_data/form/*.mpr')
datafilespath =  [os.path.basename(x) for x in datafilespath]
database = np.unique([x.split('_')[0] for x in datafilespath])

In [457]:
# query data from MySQL
##cell001, gcpl, structure 2
dbhost='127.0.0.1'
dbport=3306
dbuser='root'
dbpassword='Acakeren13!'
connection_name='SSB_Modelling'
db=database



V_Interval = [1, 1.5, 2]

for cell in db:
    Connection = pymysql.connect(host=dbhost,
                          port=dbport,
                          user=dbuser,
                          password =dbpassword,
                          database=cell)

    dfs = []
    for i in V_Interval:     
        #formation
        sql = 'SELECT `Ewe/V`, `cycle_Nr`, `Q charge/discharge/mA.h`, `half cycle` FROM {}.form_gcpl'.format(cell)
        cell_df = pd.read_sql(sql, Connection)
        cell_Qdischarge = cell_df[cell_df['half cycle'] == 1]
        cell_Qcharge = cell_df[cell_df['half cycle'] == 0]
        cycle_Nr = np.unique(cell_df['cycle_Nr'])

        QatV_form = pd.DataFrame(index=cycle_Nr)
        charge = []
        discharge = []
        charge_col = 'Q at {}V charge'.format(i)
        discharge_col = 'Q at {}V discharge'.format(i)
        for cycle in cycle_Nr:
            charge.append(cell_Qcharge['Q charge/discharge/mA.h'][(cell_Qcharge['Ewe/V'] > i) & (cell_Qcharge['cycle_Nr'] == cycle)].min())
            discharge.append(cell_Qdischarge['Q charge/discharge/mA.h'][(cell_Qdischarge['Ewe/V'] > i) & (cell_Qdischarge['cycle_Nr'] == cycle)].min())

        QatV_form[charge_col] = charge
        QatV_form[discharge_col] = discharge


        #cycling
        sql = sql = 'SELECT `Ewe/V`, `cycle_Nr`, `Q charge/discharge/mA.h`, `half cycle` FROM {}.cyc_gcpl'.format(cell)
        cell_df = pd.read_sql(sql, Connection)
        cell_Qdischarge = cell_df[cell_df['half cycle'] == 1]
        cell_Qcharge = cell_df[cell_df['half cycle'] == 0]
        cycle_Nr = np.unique(cell_df['cycle_Nr'])
        QatV_cyc = pd.DataFrame(index=cycle_Nr)
        charge = []
        discharge = []
        charge_col = 'Q at {}V charge'.format(i)
        discharge_col = 'Q at {}V discharge'.format(i)
        for cycle in cycle_Nr:
            charge.append(cell_Qcharge['Q charge/discharge/mA.h'][(cell_Qcharge['Ewe/V'] > i) & (cell_Qcharge['cycle_Nr'] == cycle)].min())
            discharge.append(cell_Qdischarge['Q charge/discharge/mA.h'][(cell_Qdischarge['Ewe/V'] > i) & (cell_Qdischarge['cycle_Nr'] == cycle)].min())

        QatV_cyc[charge_col] = charge
        QatV_cyc[discharge_col] = discharge

        dfs.append(pd.concat([QatV_form, QatV_cyc]))

    QatV = pd.concat([dfs[0], dfs[1], dfs[2]], axis=1)
    QatV.index.name = 'cycle'
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db=cell, user=dbuser, pw=dbpassword))
    QatV.to_sql('q_at_v', engine, if_exists='replace')
    print('created table q_at_v for {}'.format(cell))
print('success!')

created table q_at_v for Cell001
created table q_at_v for Cell002
created table q_at_v for Cell003
created table q_at_v for Cell004
created table q_at_v for Cell005
created table q_at_v for Cell006
created table q_at_v for Cell007
created table q_at_v for Cell008
success!
